#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

%cd /content/drive/MyDrive/2024.03.27_미니프로젝트 3차_실습자료/

dir_path = os.getcwd(); file_list = os.listdir(dir_path)

print(dir_path); print(file_list)

/content/drive/MyDrive/2024.03.27_미니프로젝트 3차_실습자료
/content/drive/MyDrive/2024.03.27_미니프로젝트 3차_실습자료
['1.탐색적 데이터 분석_에이블러용.ipynb', 'data01_test.csv', 'data01_train.csv', 'features.csv', 'features 설명.xlsx', '2.기본 모델링_에이블러용.ipynb', 'model.h5', 'model1.h5', 'model3.h5', '3.단계별 모델링_에이블러용.ipynb']


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, SimpleRNN, LSTM
from keras.backend import clear_session
from keras.optimizers import Adam
from keras.layers import Input, Flatten
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
data = pd.read_csv('data01_train.csv')
new_data = pd.read_csv('data01_test.csv')

In [ ]:
display(data.head(3))
display(new_data.head(3))

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,21,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,15,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,11,STANDING


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.284379,-0.021981,-0.116683,-0.99249,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,22,SITTING
1,0.277440,-0.028086,-0.118412,-0.99662,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,15,STANDING
2,0.305833,-0.041023,-0.087303,0.00688,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,22,WALKING


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
data['Activity'].unique()

array(['STANDING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS', 'SITTING'], dtype=object)

In [ ]:
data['Activity_dynamic'] = np.where((data['Activity']=='STANDING')|
                                    (data['Activity']=='LAYING')|
                                    (data['Activity']=='SITTING'),0,1)

new_data['Activity_dynamic'] = np.where((new_data['Activity']=='STANDING')|
                                    (new_data['Activity']=='LAYING')|
                                    (new_data['Activity']=='SITTING'),0,1)

In [ ]:
data.head(5)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,Activity_dynamic
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,21,STANDING,0
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,15,LAYING,0
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,11,STANDING,0
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,17,WALKING,1
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,17,WALKING_DOWNSTAIRS,1


## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
target1 = 'Activity'; target2 = 'Activity_dynamic'

x = data.drop([target1,target2],axis=1)
y1 = data.loc[:,target1]
y2 = data.loc[:,target2]

x_test = new_data.drop([target1,target2],axis=1)
y1_test = new_data.loc[:,target1]
y2_test = new_data.loc[:,target2]

In [ ]:
# 정적/동적 행동 분류를 위한 train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y2,test_size=0.1)

#### 1) 알고리즘1 :

In [ ]:
x_train.shape

(5292, 562)

In [ ]:
clear_session()

input = Input(shape=(562,))
ol = Dense(1,activation='sigmoid')(input)

model_1 = Model(inputs=input, outputs=ol)

model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 562)]             0         
                                                                 
 dense (Dense)               (None, 1)                 563       
                                                                 
Total params: 563 (2.20 KB)
Trainable params: 563 (2.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 컴파일
model_1.compile(optimizer=Adam(0.001), loss='binary_crossentropy')
# EarlyStopping 설정
min_de = 0.001; pat = 3
es = EarlyStopping(monitor = 'val_loss', min_delta = min_de, patience = pat)
# 학습
history = model_1.fit(x_train,y_train,epochs=100,validation_split=0.2,callbacks=[es]).history

Epoch 1/100
133/133 [==============================] - 2s 4ms/step - loss: 0.1134 - val_loss: 0.0308
Epoch 2/100
133/133 [==============================] - 0s 3ms/step - loss: 0.0198 - val_loss: 0.0158
Epoch 3/100
133/133 [==============================] - 0s 3ms/step - loss: 0.0113 - val_loss: 0.0105
Epoch 4/100
133/133 [==============================] - 0s 4ms/step - loss: 0.0078 - val_loss: 0.0085
Epoch 5/100
133/133 [==============================] - 0s 3ms/step - loss: 0.0058 - val_loss: 0.0064
Epoch 6/100
133/133 [==============================] - 0s 3ms/step - loss: 0.0046 - val_loss: 0.0055
Epoch 7/100
133/133 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0048
Epoch 8/100
133/133 [==============================] - 1s 4ms/step - loss: 0.0033 - val_loss: 0.0042
Epoch 9/100
133/133 [==============================] - 1s 5ms/step - loss: 0.0028 - val_loss: 0.0038
Epoch 10/100
133/133 [==============================] - 1s 8ms/step - loss: 0.0025 - val_lo

In [ ]:
pred = model_1.predict(x_val)
pred = np.where(pred>=0.5,1,0)

print(confusion_matrix(y_val,pred))
print(classification_report(y_val,pred))

19/19 [==============================] - 0s 2ms/step
[[300   0]
 [  0 289]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       300
           1       1.00      1.00      1.00       289

    accuracy                           1.00       589
   macro avg       1.00      1.00      1.00       589
weighted avg       1.00      1.00      1.00       589



In [ ]:
pred = model_1.predict(x_test)
pred = np.where(pred>=0.5,1,0)

print(confusion_matrix(y2_test,pred))
print(classification_report(y2_test,pred))

46/46 [==============================] - 0s 2ms/step
[[833   0]
 [  0 638]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       833
           1       1.00      1.00      1.00       638

    accuracy                           1.00      1471
   macro avg       1.00      1.00      1.00      1471
weighted avg       1.00      1.00      1.00      1471



In [ ]:
# model_1.save('model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### 2) 알고리즘2 :

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
static_data = data.loc[data['Activity_dynamic']==0]
static_new_data = new_data.loc[new_data['Activity_dynamic']==0]

static_data['Activity_num'] = static_data['Activity'].map({'STANDING': 0, 'LAYING' : 1, 'SITTING' : 2})
static_new_data['Activity_num'] = static_new_data['Activity'].map({'STANDING': 0, 'LAYING' : 1, 'SITTING' : 2})

<ipython-input-52-b96c67e6a7ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_data['Activity_num'] = static_data['Activity'].map({'STANDING': 0, 'LAYING' : 1, 'SITTING' : 2})
<ipython-input-52-b96c67e6a7ea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_new_data['Activity_num'] = static_new_data['Activity'].map({'STANDING': 0, 'LAYING' : 1, 'SITTING' : 2})


In [ ]:
static_data['Activity_num'].unique(), static_new_data['Activity_num'].unique()

(array([0, 1, 2]), array([2, 0, 1]))

In [ ]:
target1 = 'Activity'; target2 = 'Activity_dynamic'; target3 = 'Activity_num'

x = static_data.drop([target1,target2,target3],axis=1)
y1 = static_data.loc[:,target1]
y2 = static_data.loc[:,target3]

x_test = static_new_data.drop([target1,target2,target3],axis=1)
y1_test = static_new_data.loc[:,target1]
y2_test = static_new_data.loc[:,target3]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y2,test_size=0.1)

In [ ]:
# 모델 선언
# clear_session()

input = Input(shape=(562,))
hl1  = Dense(150,activation='relu')(input)
hl2  = Dense(75,activation='relu')(hl1)
hl3  = Dense(15,activation='relu')(hl2)
ol = Dense(3,activation='sigmoid')(hl3)

model_2 = Model(inputs=input, outputs=ol)

model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 562)]             0         
                                                                 
 dense_1 (Dense)             (None, 150)               84450     
                                                                 
 dense_2 (Dense)             (None, 75)                11325     
                                                                 
 dense_3 (Dense)             (None, 15)                1140      
                                                                 
 dense_4 (Dense)             (None, 3)                 48        
                                                                 
Total params: 96963 (378.76 KB)
Trainable params: 96963 (378.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 컴파일
model_2.compile(optimizer=Adam(0.0005), loss='sparse_categorical_crossentropy')
# EarlyStopping 설정
min_de = 0.0001; pat = 10
es = EarlyStopping(monitor = 'val_loss', min_delta = min_de, patience = pat)
# 학습
history = model_2.fit(x_train,y_train,epochs=100,validation_split=0.2,callbacks=[es]).history

Epoch 1/100
73/73 [==============================] - 2s 7ms/step - loss: 0.7626 - val_loss: 0.4415
Epoch 2/100
73/73 [==============================] - 0s 4ms/step - loss: 0.3314 - val_loss: 0.2526
Epoch 3/100
73/73 [==============================] - 0s 4ms/step - loss: 0.2485 - val_loss: 0.3496
Epoch 4/100
73/73 [==============================] - 0s 4ms/step - loss: 0.1956 - val_loss: 0.2310
Epoch 5/100
73/73 [==============================] - 0s 3ms/step - loss: 0.1579 - val_loss: 0.1679
Epoch 6/100
73/73 [==============================] - 0s 4ms/step - loss: 0.1629 - val_loss: 0.1496
Epoch 7/100
73/73 [==============================] - 0s 6ms/step - loss: 0.1212 - val_loss: 0.1618
Epoch 8/100
73/73 [==============================] - 1s 7ms/step - loss: 0.1163 - val_loss: 0.1636
Epoch 9/100
73/73 [==============================] - 1s 8ms/step - loss: 0.0967 - val_loss: 0.1045
Epoch 10/100
73/73 [==============================] - 0s 6ms/step - loss: 0.1008 - val_loss: 0.1541
Epoch 11/

In [ ]:
pred = model_2.predict(x_val)
pred = pred.argmax(axis=1)

print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))

11/11 [==============================] - 0s 3ms/step
[[105   0   2]
 [  1 108   0]
 [  6   0 102]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       107
           1       1.00      0.99      1.00       109
           2       0.98      0.94      0.96       108

    accuracy                           0.97       324
   macro avg       0.97      0.97      0.97       324
weighted avg       0.97      0.97      0.97       324



In [ ]:
pred = model_2.predict(x_test)
pred = pred.argmax(axis=1)

print(confusion_matrix(y2_test, pred))
print(classification_report(y2_test, pred))

27/27 [==============================] - 0s 1ms/step
[[284   0   3]
 [  0 290   2]
 [ 25   0 229]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       287
           1       1.00      0.99      1.00       292
           2       0.98      0.90      0.94       254

    accuracy                           0.96       833
   macro avg       0.97      0.96      0.96       833
weighted avg       0.97      0.96      0.96       833



In [ ]:
# model_2.save('model2.h5')

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
dynamic_data = data.loc[data['Activity_dynamic']==1]
dynamic_new_data = new_data.loc[new_data['Activity_dynamic']==1]

dynamic_data['Activity_num'] = dynamic_data['Activity'].map({'WALKING': 0, 'WALKING_DOWNSTAIRS' : 1, 'WALKING_UPSTAIRS' : 2})
dynamic_new_data['Activity_num'] = dynamic_new_data['Activity'].map({'WALKING': 0, 'WALKING_DOWNSTAIRS' : 1, 'WALKING_UPSTAIRS' : 2})

<ipython-input-60-b4cb11a45ed1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dynamic_data['Activity_num'] = dynamic_data['Activity'].map({'WALKING': 0, 'WALKING_DOWNSTAIRS' : 1, 'WALKING_UPSTAIRS' : 2})
<ipython-input-60-b4cb11a45ed1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dynamic_new_data['Activity_num'] = dynamic_new_data['Activity'].map({'WALKING': 0, 'WALKING_DOWNSTAIRS' : 1, 'WALKING_UPSTAIRS' : 2})


In [ ]:
dynamic_data['Activity_num'].unique(), dynamic_new_data['Activity_num'].unique()

(array([0, 1, 2]), array([0, 2, 1]))

In [ ]:
target1 = 'Activity'; target2 = 'Activity_dynamic'; target3 = 'Activity_num'

x = dynamic_data.drop([target1,target2,target3],axis=1)
y1 = dynamic_data.loc[:,target1]
y2 = dynamic_data.loc[:,target3]

x_test = dynamic_new_data.drop([target1,target2,target3],axis=1)
y1_test = dynamic_new_data.loc[:,target1]
y2_test = dynamic_new_data.loc[:,target3]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y2,test_size=0.1)

In [ ]:
# 모델 선언
# clear_session()

input = Input(shape=(562,))
hl1  = Dense(15,activation='relu')(input)
hl2  = Dense(9,activation='relu')(hl1)
ol = Dense(3,activation='sigmoid')(hl2)

model_3 = Model(inputs=input, outputs=ol)

model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 562)]             0         
                                                                 
 dense_5 (Dense)             (None, 15)                8445      
                                                                 
 dense_6 (Dense)             (None, 9)                 144       
                                                                 
 dense_7 (Dense)             (None, 3)                 30        
                                                                 
Total params: 8619 (33.67 KB)
Trainable params: 8619 (33.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 컴파일
model_3.compile(optimizer=Adam(0.0005), loss='sparse_categorical_crossentropy')
# EarlyStopping 설정
min_de = 0.0001; pat = 5
es = EarlyStopping(monitor = 'val_loss', min_delta = min_de, patience = pat)
# 학습
history = model_3.fit(x_train,y_train,epochs=100,validation_split=0.2,callbacks=[es]).history

Epoch 1/100
60/60 [==============================] - 1s 6ms/step - loss: 0.9964 - val_loss: 0.8666
Epoch 2/100
60/60 [==============================] - 0s 3ms/step - loss: 0.7430 - val_loss: 0.6612
Epoch 3/100
60/60 [==============================] - 0s 4ms/step - loss: 0.5751 - val_loss: 0.5306
Epoch 4/100
60/60 [==============================] - 0s 4ms/step - loss: 0.4581 - val_loss: 0.4065
Epoch 5/100
60/60 [==============================] - 0s 5ms/step - loss: 0.3161 - val_loss: 0.2663
Epoch 6/100
60/60 [==============================] - 0s 5ms/step - loss: 0.2071 - val_loss: 0.1811
Epoch 7/100
60/60 [==============================] - 0s 5ms/step - loss: 0.1488 - val_loss: 0.1411
Epoch 8/100
60/60 [==============================] - 0s 5ms/step - loss: 0.1180 - val_loss: 0.1160
Epoch 9/100
60/60 [==============================] - 0s 5ms/step - loss: 0.0933 - val_loss: 0.1002
Epoch 10/100
60/60 [==============================] - 0s 5ms/step - loss: 0.0799 - val_loss: 0.0909
Epoch 11/

In [ ]:
pred = model_3.predict(x_val)
pred = pred.argmax(axis=1)

print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))

9/9 [==============================] - 0s 2ms/step
[[89  0  0]
 [ 1 76  0]
 [ 0  0 99]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        89
           1       1.00      0.99      0.99        77
           2       1.00      1.00      1.00        99

    accuracy                           1.00       265
   macro avg       1.00      1.00      1.00       265
weighted avg       1.00      1.00      1.00       265



In [ ]:
pred = model_3.predict(x_test)
pred = pred.argmax(axis=1)

print(confusion_matrix(y2_test, pred))
print(classification_report(y2_test, pred))

20/20 [==============================] - 0s 2ms/step
[[227   1   0]
 [  0 195   0]
 [  2   0 213]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       228
           1       0.99      1.00      1.00       195
           2       1.00      0.99      1.00       215

    accuracy                           1.00       638
   macro avg       1.00      1.00      1.00       638
weighted avg       1.00      1.00      1.00       638



In [ ]:
# model_3.save('model3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### [선택사항] (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들어서 분류 모델 합치기

In [ ]:
test_data = pd.read_csv('data01_test.csv')
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.284379,-0.021981,-0.116683,-0.992490,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,22,SITTING
1,0.277440,-0.028086,-0.118412,-0.996620,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,15,STANDING
2,0.305833,-0.041023,-0.087303,0.006880,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,22,WALKING
3,0.276053,-0.016487,-0.108381,-0.995379,-0.983978,-0.975854,-0.995877,-0.985280,-0.974907,-0.941425,...,-0.841455,0.289548,0.079801,-0.020033,0.291898,-0.639435,-0.111998,-0.123298,8,SITTING
4,0.271998,0.016904,-0.078856,-0.973468,-0.702462,-0.869450,-0.979810,-0.711601,-0.856807,-0.920760,...,0.214219,0.010111,0.114179,-0.830776,-0.325098,-0.840817,0.116237,-0.096615,5,STANDING


In [ ]:
x = test_data.drop('Activity',axis=1)
y = test_data.loc[:,'Activity']

In [ ]:
from tqdm import tqdm

In [ ]:
def activity_analyze(x,y,model_1,model_2,model_3,return_pred=True):
    pred_1 = model_1.predict(x,verbose=0)
    pred_1 = np.where(pred_1 >= 0.5,1,0)

    pred = []
    for i in tqdm(range(x.shape[0])):
      if pred_1[i] == 0:
        pred_2 = model_2.predict(x.iloc[[i]],verbose=0)
        pred_2 = pred_2.argmax(axis=1)
        #값 변환
        pred_2 = np.where(pred_2 ==0,'STANDING',np.where(pred_2 ==1,'LAYING','SITTING'))
        pred.append(pred_2[0])
      else:
        pred_3 = model_3.predict(x.iloc[[i]],verbose=0)
        pred_3 = pred_3.argmax(axis=1)
        #값 변환
        pred_3 = np.where(pred_3 ==0,'WALKING',np.where(pred_3 == 1,'WALKING_DOWNSTAIRS','WALKING_UPSTAIRS'))
        pred.append(pred_3[0])
    print(); print('-'*65); print()
    print('Confusion_Matrix'); print(confusion_matrix(y, np.array(pred))); print()
    print('Classification_Report'); print(classification_report(y, np.array(pred)))

    if return_pred:
      return np.array(pred)



In [ ]:
activity_analyze(x,y,model_1,model_2,model_3,return_pred=True)

100%|██████████| 1471/1471 [01:42<00:00, 14.31it/s]


-----------------------------------------------------------------

Confusion_Matrix
[[290   2   0   0   0   0]
 [  0 229  25   0   0   0]
 [  0   3 284   0   0   0]
 [  0   0   0 227   1   0]
 [  0   0   0   0 195   0]
 [  0   0   0   2   0 213]]

Classification_Report
                    precision    recall  f1-score   support

            LAYING       1.00      0.99      1.00       292
           SITTING       0.98      0.90      0.94       254
          STANDING       0.92      0.99      0.95       287
           WALKING       0.99      1.00      0.99       228
WALKING_DOWNSTAIRS       0.99      1.00      1.00       195
  WALKING_UPSTAIRS       1.00      0.99      1.00       215

          accuracy                           0.98      1471
         macro avg       0.98      0.98      0.98      1471
      weighted avg       0.98      0.98      0.98      1471



array(['SITTING', 'STANDING', 'WALKING', ..., 'WALKING_DOWNSTAIRS',
       'LAYING', 'WALKING_DOWNSTAIRS'], dtype='<U18')